In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_predict, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/PAN16train.csv')
df = df.dropna()

df['id'].value_counts()

b4bb7a11967b660ecc4f4ef1cf6b6827    1000
d4c7789534da23ef33ab27b410132492    1000
611d21bbf8ba18cfedb76104c117ff76    1000
419453e20c2a73147960a6602d014b46    1000
da0d0e3527b931bb0bc6f5435003ea2a    1000
                                    ... 
07011b7672dfa64896c4c7b1b41ef95d      30
a959267774764d936d94ee292e992fa0      30
c297b3ecf103e67237c8a42659a178e3      24
19a9a1af56559b5d299fae4cbac79a0f      18
9343a615899a7fbd94403d228463f26d       2
Name: id, Length: 402, dtype: int64

## Tfidf 

In [8]:
tfidfvect = TfidfVectorizer( 
                            lowercase=False,
                            binary=False,
                            tokenizer=word_tokenize,
                            )

X = df['tweet']
y = df['age']

In [12]:
logreg = LogisticRegression()

pipe = Pipeline([("vec", tfidfvect),("cls", logreg)])

params = {"vec__analyzer": ["word", "char"], 
          "vec__ngram_range":[(1,1),(2,2), (3,3)],
         }

search = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)
search.fit(X,y)
search.best_estimator_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 16.7min finished
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
         steps=[('vec',
                 TfidfVectorizer(analyzer='char', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(3, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...w+\\b',
                                 tokenizer=<function word_tokenize at 0x0000020BA4E94158>,
                                 use_idf=True, vocabulary=None)),
                ('cls',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_

In [13]:
search.best_score_

0.42769550275669393

In [16]:
best_model = search.best_estimator_
result_opt_fs = cross_validate(best_model, X, y, cv=5, scoring=['f1_micro', 'f1_macro'])

print(result_opt_fs["test_f1_micro"])

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


KeyboardInterrupt: 

In [ ]:
search2 = RandomizedSearchCV(pipe, params, n_iter=5, cv=5, n_jobs=-1, verbose=2)
search2.fit(X,y)
search2.best_score_

In [18]:
tfidfvect = TfidfVectorizer(max_features=40000, 
                            lowercase=False,
                            analyzer='word', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

logreg = LogisticRegression()

X = df['tweet']
y = df['age']

In [20]:
pipe = Pipeline([("vec", tfidfvect),("cls", logreg)]) 
pipe.fit(X, y)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
         steps=[('vec',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0,
                                 max_features=40000, min_df=1,
                                 ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...w+\\b',
                                 tokenizer=<function word_tokenize at 0x0000020BA4E94158>,
                                 use_idf=True, vocabulary=None)),
                ('cls',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
       

In [21]:
pipe.predict(y)

array(['35-49', '35-49', '35-49', ..., '35-49', '35-49', '35-49'],
      dtype=object)

In [22]:
result = cross_validate(pipe, X, y, cv=5, scoring=['f1_micro', 'f1_macro'], return_train_score=True)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
result

{'fit_time': array([63.3538034 , 71.1544044 , 83.98481035, 81.59790683, 84.36624742]),
 'score_time': array([30.91471004, 25.09763551, 24.74091172, 34.80306935, 31.72141051]),
 'test_f1_micro': array([0.41652451, 0.43047864, 0.41785547, 0.43225535, 0.44875757]),
 'train_f1_micro': array([0.81076237, 0.80618344, 0.80596212, 0.81028376, 0.80366525]),
 'test_f1_macro': array([0.3327826 , 0.47125835, 0.44732691, 0.4075623 , 0.24827841]),
 'train_f1_macro': array([0.77345903, 0.74372136, 0.7508037 , 0.770429  , 0.7847075 ])}

In [24]:
tfidfvect = TfidfVectorizer(max_features=40000, 
                            lowercase=False,
                            analyzer='word', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
logreg = LogisticRegression(solver='lbfgs')

logreg.fit(X_train, y_train)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[[ 1029   735   674    44     0]
 [   24 10006  3068   344     0]
 [   17  2477 13922   454     0]
 [    7   995  2018  4382     0]
 [    2    40    66     5   138]]
              precision    recall  f1-score   support

       18-24       0.95      0.41      0.58      2482
       25-34       0.70      0.74      0.72     13442
       35-49       0.70      0.83      0.76     16870
       50-64       0.84      0.59      0.69      7402
       65-xx       1.00      0.55      0.71       251

    accuracy                           0.73     40447
   macro avg       0.84      0.63      0.69     40447
weighted avg       0.75      0.73      0.72     40447



## Countvec 

In [5]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/PAN15-train-preprocessed.csv')

In [7]:
df['pos'][0]

"['NOUN', 'PRON', 'VERB', 'ADP', 'PRON', 'NOUN', 'NOUN', 'CCONJ', 'AUX', 'ADV', 'ADJ', 'PUNCT', 'NUM', 'NOUN', 'NOUN', 'PUNCT', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'NOUN', 'PUNCT', 'VERB', 'NOUN', 'NOUN', 'PUNCT', 'VERB', 'NUM', 'NOUN', 'NOUN', 'PUNCT']"

In [9]:
# check analyzer
tfvect = CountVectorizer(analyzer='word',ngram_range=(2,2))
analyzer = tfvect.build_analyzer()
analyzer(df['pos'][0]) 

['noun pron',
 'pron verb',
 'verb adp',
 'adp pron',
 'pron noun',
 'noun noun',
 'noun cconj',
 'cconj aux',
 'aux adv',
 'adv adj',
 'adj punct',
 'punct num',
 'num noun',
 'noun noun',
 'noun punct',
 'punct verb',
 'verb noun',
 'noun punct',
 'punct adj',
 'adj noun',
 'noun noun',
 'noun punct',
 'punct verb',
 'verb noun',
 'noun noun',
 'noun punct',
 'punct verb',
 'verb num',
 'num noun',
 'noun noun',
 'noun punct']

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
print("Majority baseline:", 5428/len(df))

Majority baseline: 0.39944072411509307


In [5]:
# BOW baseline
vectorizer = CountVectorizer(tokenizer= word_tokenize, 
                             analyzer='word', 
                             ngram_range=(1, 1)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1: 0.4955 (+/- 0.0456)


In [9]:
vectorizer = CountVectorizer(tokenizer= lambda x: x.split(), 
                             analyzer='word', 
                             ngram_range=(1, 1)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1: 0.4684 (+/- 0.0350)


In [18]:
vectorizer = CountVectorizer(tokenizer= word_tokenize, 
                             analyzer='char', 
                             ngram_range=(1,1)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1: 0.3913 (+/- 0.0501)


In [ ]:
# lambda x: x.split: f1: 0.3913 (+/- 0.0501)
# word_tokenize: f1: 0.3913 (+/- 0.0501)

In [19]:
vectorizer = CountVectorizer(tokenizer= word_tokenize, 
                             analyzer='char', 
                             ngram_range=(2,2)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1: 0.4659 (+/- 0.0459)


In [ ]:
# lamda x: x.split: f1: 0.4659 (+/- 0.0459)
# word_tokenize: 

In [15]:
vectorizer = CountVectorizer(tokenizer= lambda x: x.split(), 
                             analyzer='char', 
                             ngram_range=(3,3)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1: 0.4830 (+/- 0.0406)


In [16]:
vectorizer = CountVectorizer(tokenizer= lambda x: x.split(), 
                             analyzer='char', 
                             ngram_range=(4,4)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1: 0.5039 (+/- 0.0499)


In [21]:
vectorizer = CountVectorizer(tokenizer= word_tokenize, 
                             analyzer='char', 
                             ngram_range=(5,5)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1: 0.5097 (+/- 0.0489)


In [20]:
vectorizer = CountVectorizer(tokenizer= lambda x: x.split(), 
                             analyzer='char', 
                             ngram_range=(6,6)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


f1: 0.5034 (+/- 0.0529)


In [ ]:
# pos - ngrams
vectorizer = CountVectorizer( 
                             analyzer='word', 
                             ngram_range=(1,1)) 

X = vectorizer.fit_transform(df['pos']) 
y = df['pos']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [ ]:
# pos - ngrams
vectorizer = CountVectorizer(
                             analyzer='word', 
                             ngram_range=(2,2)) 

X = vectorizer.fit_transform(df['pos']) 
y = df['pos']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=3) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

In [ ]:
# pos - ngrams
vectorizer = CountVectorizer(tokenizer= word_tokenize, 
                             analyzer='word', 
                             ngram_range=(3,3)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['pos']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

In [ ]:
# pos - ngrams
vectorizer = CountVectorizer(tokenizer= word_tokenize, 
                             analyzer='word', 
                             ngram_range=(4,4)) 

X = vectorizer.fit_transform(df['tweet']) 
y = df['pos']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

In [ ]:
# pos - ngrams
vectorizer = CountVectorizer(tokenizer= word_tokenize, 
                             analyzer='word', 
                             ngram_range=(5,5)) 

X = vectorizer.fit_transform(df['pos']) 
y = df['age']

clf_svc = LinearSVC(random_state=0)

clf_svc.fit(X, y)

f1 = cross_val_score(clf_svc, X, y, cv=10, scoring='f1_macro', n_jobs=-1) 
print("f1: %0.4f (+/- %0.4f)" % (f1.mean(), f1.std()))

In [11]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/PAN16train.csv')
df = df.dropna()
len(df)

246886

# combining features

In [2]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/PAN15-train-preprocessed.csv')

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

X = df['tweet']
y = df['age']

tfvect = CountVectorizer()
nb = MultinomialNB(alpha = 0.1, fit_prior = True)

pipe = Pipeline([("vec", tfvect),("feat_sel", SelectKBest(mutual_info_classif)),("cls", nb)])
params = {"feat_sel__k": [10, 100, 1000, 5000],
         }

search = GridSearchCV(pipe, params, cv=5, n_jobs=-1, verbose=1)
search.fit(X,y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 26.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vec',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                             

In [7]:
search.best_params_

{'feat_sel__k': 5000}

In [8]:
search.best_score_

0.592390904407977

In [5]:
#with logreg
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

X = df['tweet']
y = df['age']

tfvect = CountVectorizer()
logreg = LogisticRegression()

pipe = Pipeline([("vec", tfvect),("feat_sel", SelectKBest(mutual_info_classif)),("cls", logreg)])
params = {"feat_sel__k": [10, 100, 1000, 5000, 10000],
         }

search = GridSearchCV(pipe, params, cv=10, n_jobs=-1, verbose=1)
search.fit(X,y)

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 65.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 74.6min finished
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vec',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            